Importacion de librerias y .CSV con los datos

In [2]:
import pandas as pd
import itertools

data = pd.read_csv("COVID19MEXICO.csv")
data

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-07-29,167f1a,1,12,1,2,1,1,3,1,...,997,997,2,97,6,6,99,México,97,97
1,2025-07-29,ga8a474,1,4,20,2,20,20,413,1,...,997,997,2,97,6,6,99,México,97,97
2,2025-07-29,gb733da,1,6,8,2,8,8,37,1,...,997,997,2,97,6,6,99,México,97,97
3,2025-07-29,g9ff7b3,1,4,32,1,32,32,17,2,...,5,5,2,97,7,7,99,México,97,2
4,2025-07-29,g90b5c8,1,6,10,1,10,10,5,2,...,5,5,2,97,7,7,99,México,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,2025-07-29,g897fca,1,3,21,1,21,21,53,2,...,999,999,2,97,6,6,99,México,97,2
99522,2025-07-29,ge40110,1,15,16,2,15,16,102,2,...,999,999,2,97,6,6,99,México,97,2
99523,2025-07-29,g528417,1,15,2,1,2,2,1,2,...,999,999,2,97,6,6,99,México,97,2
99524,2025-07-29,gab8b98,1,12,19,2,19,19,44,1,...,999,999,2,97,6,6,99,México,97,97


# Variables que se toman en cuenta y su contexto
En el contexto de la base de datos de la Secretaría de Salud de México, los códigos de valor son:


1: Sí


2: No


97, 98, 99: Desconocido o no aplica


Edad:    Numero de la edad del paciente


NEUMONIA:	Paciente que presentó o no diagnóstico de neumonía.


DIABETES:	Paciente con un diagnóstico previo de diabetes.


EPOC:	Paciente con un diagnóstico previo de enfermedad pulmonar obstructiva crónica.


ASMA:	Paciente con un diagnóstico previo de asma.


INMUSUPR:	Paciente con un diagnóstico previo de inmunosupresión.


HIPERTENSION:	Paciente con un diagnóstico previo de hipertensión.


CARDIOVASCULAR:	Paciente con un diagnóstico previo de enfermedad cardiovascular.


OBESIDAD:	Paciente con un diagnóstico previo de obesidad.


RENAL_CRONICA:	Paciente con un diagnóstico previo de enfermedad renal crónica.


TABAQUISMO:	Paciente con un diagnóstico previo de tabaquismo.


OTRO_CASO:	Indica si el paciente tuvo contacto con otro caso confirmado de COVID-19.


FECHA_DEF: Contiene la fecha del fallecimiento del paciente en caso de tener 9999-99-99 o algo parecido no ha muerto



Y la más Importante


### CLASIFICACION_FINAL_COVID


3: Negativo a SARS-CoV-2: El caso fue descartado tras obtener un resultado de laboratorio negativo.


4: Caso Descartado por Dictaminación: El caso fue descartado por un comité de expertos sin una prueba de laboratorio concluyente.


5: Sospechoso: Es el estado inicial y transitorio de un caso, cuya clasificación final está pendiente de confirmación o descarte.


6: Confirmado por Laboratorio: El caso fue confirmado mediante una prueba de laboratorio, típicamente RT-PCR, el método de mayor fiabilidad.


7: Positivo por Dictaminación o Asociación: El caso fue catalogado como positivo por un comité de expertos basándose en un nexo epidemiológico con un caso confirmado o en criterios clínicos, sin una prueba de laboratorio concluyente.

### CLASIFICACION_FINAL_FLU

(3): Caso confirmado


(7): Caso negativo


(4, 5 ó 6): Sin información (sin muestra o muestra inválida)

# Limpieza

In [3]:
# Seleccionando las columnas de interes
data = data[[
    "EDAD",
    "NEUMONIA",
    "DIABETES",
    "EPOC",
    "ASMA",
    "INMUSUPR",
    "HIPERTENSION",
    "CARDIOVASCULAR",
    "OBESIDAD",
    "RENAL_CRONICA",
    "TABAQUISMO",
    "OTRO_CASO",
    "FECHA_DEF",
    "CLASIFICACION_FINAL_COVID",
    "CLASIFICACION_FINAL_FLU"
]].copy()
# Lista de valores que representan datos faltantes
missing_codes = [97, 99, 997, 999, 9999, '97', '99', '997', '999', '9999', '9999-99-99']

# Reemplazar por 0 en todo el DataFrame
data = data.replace(missing_codes, 0)

# Crear una columana adulto mayor para centrarse solo en esa la probabilidad 
adulto_mayor = data["EDAD"].apply(
    lambda x: 1 if pd.notna(x) and x >= 60 else 2) #se toma 60 que es la edad para ser adulto mayor en México
data.insert(1, 'AM', adulto_mayor)

# Esta funcion es para en lugar de tener fecha de muerte, tener si se murio o no
def convertir_no_cero_a_uno(valor):
    """Convierte valores no cero a 1, mantiene ceros como 2"""
    return 1 if valor != 0 else 2

muertos = data["FECHA_DEF"]
data["FECHA_DEF"] = muertos.apply(convertir_no_cero_a_uno)

def crear_columna_enfermo(df):
    df = df.copy()
    
    df['ENFERMO'] = (
        (df['CLASIFICACION_FINAL_COVID'].isin([6, 7])) |  # COVID positivo
        (df['CLASIFICACION_FINAL_FLU'] == 3)              # Influenza positivo
    )
    # Igual usamos 1 para casos positivos y 2 para negativos
    df['ENFERMO'] = df['ENFERMO'].map({True: 1, False: 2})
    
    return df
data = crear_columna_enfermo(data)

data

,EDAD,AM,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,FECHA_DEF,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,ENFERMO
0,8,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
1,23,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
2,18,2,2,2,2,1,2,2,2,2,2,2,2,2,6,6,1
3,24,2,2,2,2,1,2,2,2,2,2,2,2,2,7,7,1
4,47,2,2,2,2,2,2,2,2,2,2,2,2,2,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,83,1,2,2,2,2,2,1,2,2,2,2,2,2,6,6,1
99522,50,2,1,1,2,2,2,1,2,2,1,2,1,2,6,6,1
99523,13,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
99524,37,2,2,2,2,2,2,2,2,2,2,2,1,2,6,6,1


# Algunos porcentajes importantes

In [4]:
s = data["ENFERMO"].value_counts()
total_poblacion = data["ENFERMO"].count()
print(f"{s}, total de población: {total_poblacion} ")
sintomas = ["AM", "NEUMONIA", "DIABETES", "EPOC", "ASMA", 
            "INMUSUPR", "HIPERTENSION", "CARDIOVASCULAR", 
            "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO", "OTRO_CASO"]

# Filtrar pacientes muertos
muertos = data[data['FECHA_DEF'] == 1]

# Contar cuántos tienen todos los síntomas = 0
todos_0 = muertos[(muertos[sintomas] == 0).all(axis=1)]
num_todos_0 = len(todos_0)

# Contar cuántos tienen todos los síntomas = 2
todos_2 = muertos[(muertos[sintomas] == 2).all(axis=1)]
num_todos_2 = len(todos_2)

print(f"Muertos los cuales no sabemos ninguno de sus sintomas: {num_todos_0}")
print(f"Muertos asintomaticos: {num_todos_2}")
muertos = 0
for i in range(len(data["FECHA_DEF"])):
    if data["FECHA_DEF"][i] == 1:
        muertos +=1
print(f"Total de muertos: {muertos}")
Pct_muertos = muertos/99526
print(f"Porcentaje  de muertos: {round(Pct_muertos*100,2)}%")

ENFERMO
1    92054
2     7472
Name: count, dtype: int64, total de población: 99526 
Muertos los cuales no sabemos ninguno de sus sintomas: 0
Muertos asintomaticos: 279
Total de muertos: 3710
Porcentaje  de muertos: 3.73%


In [5]:
for sintoma in sintomas:
    # Contar cuantas veces coincide el sintoma con que este enfermo el paciente
    sintoma_y_enfermo = len(data[(data[sintoma] == 1) & (data['ENFERMO'] == 1)])
    total_con_sintoma = len(data[data[sintoma] == 1])
    
    #Calcular cuantas personas enfermas tienen el sintoma
    pct_enfermos_con_sintoma = (sintoma_y_enfermo / total_poblacion) * 100
    # Calcular de las personas con el sintoma cuantos estan enfermos
    pct_sintoma_que_son_enfermos = (sintoma_y_enfermo / total_con_sintoma) * 100
    s
        
    if sintoma == sintomas[0]:
        print(f"{'Síntoma':15} {'% ' 'Enfermos con síntoma en la población total':>25} {' % ' 'Si tiene sintoma esta enfermo ':>30}")
    print(f"{sintoma:15} {pct_enfermos_con_sintoma:25.2f}% {pct_sintoma_que_son_enfermos:40.2f}%")

Síntoma         % Enfermos con síntoma en la población total  % Si tiene sintoma esta enfermo 
AM                                  19.69%                                    91.15%
NEUMONIA                            21.95%                                    92.90%
DIABETES                            12.12%                                    91.69%
EPOC                                 3.11%                                    92.46%
ASMA                                 3.67%                                    93.23%
INMUSUPR                             2.84%                                    89.64%
HIPERTENSION                        14.62%                                    91.34%
CARDIOVASCULAR                       3.11%                                    92.25%
OBESIDAD                             6.24%                                    92.09%
RENAL_CRONICA                        3.17%                                    91.24%
TABAQUISMO                           4.15%             

In [6]:
# Calcular y mostrar % de muertos por síntoma (sobre población total y sobre quienes tienen el síntoma)
resultados_top_sintomas_mortiferos = []
for sintoma in sintomas:
    total_con_sintoma = len(data[data[sintoma] == 1])
    sintoma_y_muerto = len(data[(data[sintoma] == 1) & (data['FECHA_DEF'] == 1)])
    
    if sintoma == sintomas[0]:
        print(f"{'Síntoma':15} {'% Muertos que presentaron el sintoma (sobre población total)':>45} {'% Peronas con sintoma que estan muertos':>55}")
    
    pct_muertos_con_sintoma = (sintoma_y_muerto / total_poblacion) * 100
    pct_sintoma_que_estanmuertos = (sintoma_y_muerto / total_con_sintoma * 100) if total_con_sintoma > 0 else 0.0
    
    print(f"{sintoma:15} {pct_muertos_con_sintoma:25.2f}% {pct_sintoma_que_estanmuertos:60.2f}%")
    
    resultados_top_sintomas_mortiferos.append({
        'sintoma': sintoma,
        'pct_muertos_entre_con_sintoma': pct_sintoma_que_estanmuertos,
        'muertos': sintoma_y_muerto,
        'total_con_sintoma': total_con_sintoma
    })

# Ordenar de mayor a menor y mostrar top 5
top5 = sorted(resultados_top_sintomas_mortiferos, key=lambda x: x['pct_muertos_entre_con_sintoma'], reverse=True)[:5]
print("\nTop 5 síntomas por % de quienes los tienen que murieron (de mayor a menor):")
for i, r in enumerate(top5, start=1):
    print(f"{i}. {r['sintoma']}: {r['pct_muertos_entre_con_sintoma']:.2f}% ({r['muertos']}/{r['total_con_sintoma']})")

Síntoma         % Muertos que presentaron el sintoma (sobre población total)                 % Peronas con sintoma que estan muertos
AM                                   2.16%                                                        10.00%
NEUMONIA                             2.44%                                                        10.32%
DIABETES                             1.25%                                                         9.49%
EPOC                                 0.41%                                                        12.20%
ASMA                                 0.09%                                                         2.25%
INMUSUPR                             0.26%                                                         8.08%
HIPERTENSION                         1.43%                                                         8.93%
CARDIOVASCULAR                       0.42%                                                        12.49%
OBESIDAD                   

Los 5 sintomas más mortiferos y la probabilidad de tenerlo


AM = https://www.gob.mx/inapam/articulos/proyecciones-demograficas-de-un-mexico-que-envejece


Neumonia = No se encontraron datos exactos, se va a tener que calcular en base a los datos obtenidos


EPOC = https://www.gob.mx/salud/articulos/sabias-que-10-de-la-poblacion-mexicana-padece-epoc


CARDIOVASCULAR =  No se encontraron datos exactos, se va a tener que calcular en base a los datos obtenidos


RENAL CRONICA = https://www.gaceta.unam.mx/casi-12-de-la-poblacion-sufre-enfermedad-renal-cronica-en-mexico/



In [7]:
def calculo_sintoma(df,sintoma):
    total_con_sintoma = len(df[df[sintoma] == 1])
    Pct_Sintoma = (total_con_sintoma/total_poblacion)*100
    return Pct_Sintoma

# La probabilidad de tener cada sintoma
Probabilidad_AM = 12.8
Probabilidad_Neumonia = round(calculo_sintoma(data,"NEUMONIA"),2) #23.63
Probabilidad_EPOC = 10
Probabilidad_Cardiovascular = round(calculo_sintoma(data,"CARDIOVASCULAR"),2) #3.37
Probabilidad_Renal_Cronica = 12

Ndata = {
    'Sintoma': ["AM", "NEUMONIA", "EPOC", "CARDIOVASCULAR", "RENAL_CRONICA"],
    'Probabilidad_Enfermedad': [0.128, 0.2363, 0.1, .337, 0.12],
    'Pct_de_enfermos_con_sintoma': [0.2129, 0.2373, 0.336, 0.336, 0.343],
    'Pct_de_persomas_con_sintoma_Enfermos': [0.9115, 0.929, 0.9246, 0.9225, 0.9124]
}

dfNdata = pd.DataFrame(Ndata)
dfNdata

,Sintoma,Probabilidad_Enfermedad,Pct_de_enfermos_con_sintoma,Pct_de_persomas_con_sintoma_Enfermos
0,AM,0.1280,0.2129,0.9115
1,NEUMONIA,0.2363,0.2373,0.9290
2,EPOC,0.1000,0.3360,0.9246
3,CARDIOVASCULAR,0.3370,0.3360,0.9225
4,RENAL_CRONICA,0.1200,0.3430,0.9124


# Relaciones lógicas ruidosas
En redes bayesianas, una **relación lógica ruidosa** se utiliza cuando una variable depende de varios padres mediante una operación lógica, pero se introduce **incertidumbre** sobre la capacidad de cada padre de causar el evento en el hijo.  

En nuestro caso es el **noisy-OR**, una generalización del operador OR:

- En lógica proposicional,(ejemplo) diríamos que un paciente tiene **EPOC** si y solo si tiene **Neumonia**, Enfermedades **Cardiovasculares** o **Renales**.  
- En el modelo noisy-OR, cada causa (AM, NEUMONIA, EPOC, CARDIOVASCULAR, RENAL) puede contribuir al resultado positivo del (Diagnóstico), pero con una cierta **probabilidad de inhibición** que refleja que, aun si la causa está presente, puede no producir el efecto.  

---

## Suposiciones del modelo noisy-OR

1. **Se enumeran todas las causas posibles.**  
   Si faltan causas, se puede introducir un *leak node* (nodo de fuga) para representarlas.  

2. **La inhibición de cada causa es independiente de las demás.**  
   Es decir, que lo que evita que “EPOC” cause un diagnóstico positivo no influye en lo que evita que “Neumonía” la cause.  

---

## Fórmula para calcular la CPT (Noisy-OR)
$$
\text{Sea } x_i \text{ el efecto y } \mathrm{Padres}(X_i)=\{X_1,\dots,X_k\} \text{ sus causas.}
$$
Cada causa \(j\) tiene una probabilidad de inhibición \(q_j\), definida como
$$
q_j \;=\; P\big(\neg x_i \mid X_j = 1,\; X_{m\neq j}=0\big),
$$
es decir, la probabilidad de que la causa \(j\) por sí sola no produzca el efecto.

El modelo noisy-OR establece entonces:
$$
P\big(x_i \mid X_1,\dots,X_k\big) \;=\; 1 \;-\; \prod_{j:\,X_j=1} q_j.
$$


# Caclular P(S) 

que son 2^5 posibilidades dado a que tenemos 5 sintomas y 2 posibildades (Tenerlo y no tenerlo)

In [8]:
def p_s(AM: bool, NEUMONIA: bool , EPOC: bool, CARDIOVASCULAR: bool, RENAL_CRONICA: bool, dfData):
    if AM == True:
        Valor_am = dfData["Probabilidad_Enfermedad"][0]
    else:
        Valor_am = 1 - dfData["Probabilidad_Enfermedad"][0]
    if NEUMONIA == True:
        Valor_NEUMONIA = dfData["Probabilidad_Enfermedad"][1]
    else:
        Valor_NEUMONIA = 1 - dfData["Probabilidad_Enfermedad"][1]
    if EPOC == True:
        Valor_EPOC= dfData["Probabilidad_Enfermedad"][2]
    else:
        Valor_EPOC = 1 - dfData["Probabilidad_Enfermedad"][2]
    if CARDIOVASCULAR == True:
        Valor_CARDIOVASCULAR = dfData["Probabilidad_Enfermedad"][3]
    else:
        Valor_CARDIOVASCULAR = 1 - dfData["Probabilidad_Enfermedad"][3]
    if RENAL_CRONICA == True:
        Valor_RENAL_CRONICA = dfData["Probabilidad_Enfermedad"][4]
    else:
        Valor_RENAL_CRONICA = 1 - dfData["Probabilidad_Enfermedad"][4]
    p_s = Valor_am*Valor_NEUMONIA*Valor_EPOC*Valor_CARDIOVASCULAR*Valor_RENAL_CRONICA
    return p_s

Probabilidad_S = p_s(True,True,True,True,True, dfNdata)
print(f"P(D) = {Probabilidad_S}")
combinaciones = list(itertools.product([False, True], repeat=5))

# Crear tabla con resultados
tabla = []
for comb in combinaciones:
    prob = p_s(*comb, dfNdata)
    fila = dict(zip(dfNdata["Sintoma"], comb))
    fila["Probabilidad_S"] = prob
    tabla.append(fila)

df_resultados = pd.DataFrame(tabla)
df_resultados

P(D) = 0.0001223164416


,AM,NEUMONIA,EPOC,CARDIOVASCULAR,RENAL_CRONICA,Probabilidad_S
0,False,False,False,False,False,0.349686
1,False,False,False,False,True,0.047684
2,False,False,False,True,False,0.177744
3,False,False,False,True,True,0.024238
4,False,False,True,False,False,0.038854
5,False,False,True,False,True,0.005298
6,False,False,True,True,False,0.019749
7,False,False,True,True,True,0.002693
8,False,True,False,False,False,0.108198
9,False,True,False,False,True,0.014754


# Calculos de P(¬S|D) y P(S|D)

In [9]:
resultados = []

# Primero calculamos el total de muertes para los porcentajes
total_enfermos = len(data[data['ENFERMO'] == 1])

for comb in itertools.product([0, 1], repeat=len(dfNdata["Sintoma"])):
    # Construir la query string
    condiciones_query = []
    for i, col in enumerate(dfNdata["Sintoma"]):
        if comb[i] == 1:
            condiciones_query.append(f"{col} == 1")
        else:
            condiciones_query.append(f"{col} != 1")
    
    query_str = " & ".join(condiciones_query) + " & ENFERMO == 1"
    
    # Calcular el conteo
    conteo = len(data.query(query_str))
    
    # Calcular porcentaje
    porcentaje = (conteo / total_enfermos * 100) if total_enfermos > 0 else 0
    
    # Crear diccionario con resultados
    resultado = {col: comb[i] for i, col in enumerate(dfNdata["Sintoma"])}
    resultado['ENFERMOS'] = conteo
    resultado['PORCENTAJE'] = round(porcentaje, 2)
    resultados.append(resultado)

# Crear DataFrame final
tabla_resultados = pd.DataFrame(resultados)

print(tabla_resultados)
print(f"\nTotal de enfermos: {total_enfermos}")

    AM  NEUMONIA  EPOC  CARDIOVASCULAR  RENAL_CRONICA  ENFERMOS  PORCENTAJE
0    0         0     0               0              0     56475       61.35
1    0         0     0               0              1       763        0.83
2    0         0     0               1              0       477        0.52
3    0         0     0               1              1        74        0.08
4    0         0     1               0              0       249        0.27
5    0         0     1               0              1        16        0.02
6    0         0     1               1              0        21        0.02
7    0         0     1               1              1        15        0.02
8    0         1     0               0              0     13107       14.24
9    0         1     0               0              1       508        0.55
10   0         1     0               1              0       414        0.45
11   0         1     0               1              1        69        0.07
12   0      

El primero que no incluye P(¬S|D) siendo 61.35% y P(S(i)|D) es el resto

# Calculo de P(D | Sintomas) y P(D | ¬Sintomas)

Para eso obtenemos para cada P(S(i)) buscamos cuantos enfermos tiene y lo dividimos entre su población total

In [10]:
resultados = []

total_poblacion = len(data)
total_enfermos = len(data[data['ENFERMO'] == 1])

for comb in itertools.product([0, 1], repeat=len(dfNdata["Sintoma"])):
    # Construir la query base (síntomas)
    condiciones_query = []
    for i, col in enumerate(dfNdata["Sintoma"]):
        if comb[i] == 1:
            condiciones_query.append(f"{col} == 1")
        else:
            condiciones_query.append(f"{col} != 1")
    
    query_str = " & ".join(condiciones_query)
    
    # Conteos
    conteo_poblacion = len(data.query(query_str))
    conteo_enfermos = len(data.query(query_str + " & ENFERMO == 1"))
    
    # Probabilidad condicional ENFERMO|S
    if conteo_poblacion > 0:
        prob_enfermo_dado_s = conteo_enfermos / conteo_poblacion
    else:
        prob_enfermo_dado_s = 0
    
    # Crear diccionario con resultados
    resultado = {col: comb[i] for i, col in enumerate(dfNdata["Sintoma"])}
    resultado['P(D|S)'] = round(prob_enfermo_dado_s, 4)  # redondeado a 4 decimales
    resultados.append(resultado)

# Crear DataFrame final
tabla_resultados = pd.DataFrame(resultados)

print(tabla_resultados)

    AM  NEUMONIA  EPOC  CARDIOVASCULAR  RENAL_CRONICA  P(D|S)
0    0         0     0               0              0  0.9277
1    0         0     0               0              1  0.8987
2    0         0     0               1              0  0.9138
3    0         0     0               1              1  0.8810
4    0         0     1               0              0  0.9188
5    0         0     1               0              1  0.8000
6    0         0     1               1              0  0.8750
7    0         0     1               1              1  0.9375
8    0         1     0               0              0  0.9362
9    0         1     0               0              1  0.8975
10   0         1     0               1              0  0.9495
11   0         1     0               1              1  0.8734
12   0         1     1               0              0  0.9447
13   0         1     1               0              1  0.9474
14   0         1     1               1              0  0.9787
15   0  

P(D|¬S) = 0.9277 algo alta pero, los 5 sintomas solo representan el 39% de la población y al ser basados parea mortandad, pues los niveles de enfremedades de los otros sintomas son altos, pero no tienen un impacto tan alto en las muertes por malos diagnosticos, además el calculo se basa en el dataset que tenemos, el cual se puede ver sesgado ya que solo tenemos personas que van al medico y que seguramente presentan algun tipo de molestia, pero en en este caso creemos que si sirve porque ya que una persona que no presenta molestias no se presentaria en el medico, el resto son es P(D|S(i))

# Ahora calcularemos P(D)

 **5 síntomas** ($S_1, S_2, S_3, S_4, S_5$), queda:
$$
P(D) = \sum_{s_1=0}^1 \sum_{s_2=0}^1 \sum_{s_3=0}^1 \sum_{s_4=0}^1 \sum_{s_5=0}^1 P(D \mid s_1, s_2, s_3, s_4, s_5)\, P(s_1)\,P(s_2)\,P(s_3)\,P(s_4)\,P(s_5),
$$

 **Todas las combinaciones posibles de síntomas (32 combinaciones)**, multiplicas la probabilidad condicional $P(D \mid S)$ por las probabilidades de cada síntoma, y luego sumas.


In [15]:
def p_d(pDdadoS,pS):
    Valor_pd = 0
    for i in range(len(pS)):
        Valor_pd += pDdadoS[i]*pS[i]
    return Valor_pd

P_d = round((p_d(tabla_resultados['P(D|S)'], df_resultados["Probabilidad_S"])),4)
P_Nod = round(1 - (p_d(tabla_resultados['P(D|S)'], df_resultados["Probabilidad_S"])),4)

print(f"P(D) = {P_d} y P(¬D): {P_Nod}")

P(D) = 0.9206 y P(¬D): 0.0794


Por ende P(D) = 92.06% y y P(¬D) = 7.94%

### Calculo P(D|M) y P(¬D|M)

In [12]:
# Casos con D y M
casos_DM = len(data[(data["ENFERMO"] == 1) & (data["FECHA_DEF"] == 1)])

# Calculo de probabilidades
P_D_dado_M = casos_DM / muertos
P_notD_dado_M = 1 - P_D_dado_M

print(f"P(D|M) = {P_D_dado_M}")
print(f"P(¬D|M) = {P_notD_dado_M}")


P(D|M) = 0.8981132075471698
P(¬D|M) = 0.10188679245283017


Falsos negativos 10.18%

# Calculo P(M | ¬D)

In [13]:
P_M_dado_notD = (P_notD_dado_M*Pct_muertos) / P_Nod
print(f"P(M|¬D) = {P_M_dado_notD}")

P(M|¬D) = 0.047833785037804624


Calculo de error de diagnostico, muertos con mal diagnostico 4.78%

# Calculo P(D∣¬M) falsos positivos

In [14]:
Muerto_ySiDiagnostico = 

SyntaxError: invalid syntax (4236319537.py, line 1)

## Referencias

Russell, S., & Norvig, P. (2021). *Artificial intelligence: A modern approach* (4th ed., pp. 510–519). Pearson.
Huang, B. (2015, [25 de Mayo]). *Bayesian Networks* [Video]. YouTube. https://www.youtube.com/watch?v=TuGDMj43ehw
